In [1]:
import pandas as pd
import requests
import bs4 as BeautifulSoup
import json
from dotenv import load_dotenv
import os

In [2]:
# Lưu api key vào file .env để bảo mật
load_dotenv()
api_key = os.getenv('API_KEY')
# url mặc định
base_url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'
# khu vực đông nam bộ
location1 = 'Ho Chi Minh'
# khu vực đồng bằng sông cửu long (tây nam bộ, hay miền tây)
location2 = 'Ben Tre'
# Do mỗi ngày chỉ crawl được khoảng 1000 ngày nên em chia ra 1 khu vực crawl trong 2 ngày
# Nửa đầu
date1 = '2019-01-01'
date2 = '2021-06-30'
# Nửa sau
date3 = '2021-07-01'
date4 = '2024-01-01'

In [3]:
def getData(location: str, date1: str, date2: str) -> dict:
    url = f"{base_url}{location}/{date1}/{date2}"
    params = {
        "key": api_key,
        "unitGroup": "metric",  # Sử dụng độ C
        "include": "days",   # Lấy dữ liệu thời tiết hiện tại
        "contentType": "json",  # Định dạng trả về là JSON
        "lang" : "en"
    }
    response = requests.get(url, params=params)
    # Nếu status code khác 200 -> lỗi khi gọi api
    assert response.status_code == 200, f'Error: {response.status_code}\n{response.content}'
    return response.json()

# Lưu file json phòng trường hợp cần dùng lại mà không cần phải gọi lại api
def saveJson(data: dict, fileName: str):
    with open(fileName, 'w') as f:
        f.write(str(data).replace('\'', '"') # đổi các cặp ngoặc '' thành "" 
                         .replace('None', '""')) # đổi các mục không có data từ None thành ""

# Đọc file json
def readJson(fileName: str):
    with open(fileName, 'r') as f:
        return json.load(f)

In [4]:
def up1stChar(s: str) -> str: # ghi hoa ký tự đầu
    return s[0].upper() + s[1:]

def convertDataToDf(data: dict) -> pd.DataFrame: # chuyển data file json qua dataframe
    df_header = list(data['days'][0].keys())
    df_header
    dict_data = {
        up1stChar(header): [] for header in df_header
    }
    for d in data['days']:
        for header in df_header:
            dict_data[up1stChar(header)].append(str(d[header])
                                        .replace('[', '')
                                        .replace(']', '')
                                        .replace('\'', '')) # bỏ các dấu ', [, ] trong list khi chuyển qua str
    dict_data['Address'] = [data['address']] * len(data['days'])

    df = pd.DataFrame(dict_data)
    return df

In [5]:
# Ngày 1: gọi api lấy data location1, date1 -> date2
data = getData(location1, date1, date2)
saveJson(data, f'{location1} (1).json')

In [5]:
# Ngày 2: gọi api lấy data location1, date3 -> date4
data = getData(location1, date3, date4)
saveJson(data, f'{location1} (2).json')

In [6]:
# Ngày 2: merge data location1 của 2 ngày lại
oldData = readJson(f'{location1} (1).json')
data = readJson(f'{location1} (2).json')

df1 = convertDataToDf(oldData)
df2 = convertDataToDf(data)

df = pd.concat([df1, df2], axis=0, ignore_index=True)
df.to_csv(f'{location1}.csv')

In [5]:
# Ngày 3: gọi api lấy data location2, date1 -> date2
data = getData(location2, date1, date2)
saveJson(data, f'{location2} (1).json')

In [ ]:
# Ngày 4: gọi api lấy data location2, date3 -> date4
data = getData(location2, date3, date4)
saveJson(data, f'{location2} (2).json')

In [ ]:
# Ngày 4: merge data location2 của 2 ngày lại
oldData = readJson(f'{location2} (1).json')
data = readJson(f'{location2} (2).json')

df1 = convertDataToDf(oldData)
df2 = convertDataToDf(data)

df = pd.concat([df1, df2], axis=0, ignore_index=True)
df.to_csv(f'{location2}.csv')